# Fixing triangular meshes with trimesh

Consider: 
* The function `txttoarray` will only work for txt files inside the folder this notebook is in.
* Mesh will be made watertight by filling holes, deleting repeted vertices and faces, and by merging vertices that are close together. This distance can be toggled through the variable `merge_tolerance`.

In [1]:
%reset -f
import time
import numpy as np
import trimesh


def txttoarray(filename):
    file = open(filename, "r").read().split("\n")
    datos=[]
    for i in range(len(file)):
        datos.append(file[i].split())
    return(datos)

######################### #########################
######################### #########################

toc = time.time()
    
# Replace "surf_d02_stern.face" and "vert" with filename (txt extention)
face=txttoarray("Meshing_1fni/1fni_d16.face")
face=[i for i in face if i!=[]]

vert=txttoarray("Meshing_1fni/1fni_d16.vert")
vert=[i for i in vert if i!=[]]

vert_txt=np.float_(vert) ; vert_norm=vert_txt[:,3:6] ; vert=vert_txt[:,:3]
face=np.int_(face); face= face[:,:3]-1

######################### #########################
######################### #########################
    
mesh = trimesh.Trimesh(vertices=vert,faces=face,vertex_normals=vert_norm,validate=True,process=True)
mesh.fill_holes()
mesh.process()

while not mesh.is_watertight:
    merge_tolerance = 0.005
    needy_faces     = trimesh.repair.broken_faces(mesh)
    for vert_nf in mesh.faces[needy_faces]:
        for nf in vert_nf:
            for c,check in enumerate(np.linalg.norm(mesh.vertices[vert_nf]-mesh.vertices[nf],axis=1)):
                if (check<merge_tolerance) & (0<check):
                    mesh.vertices[nf]=mesh.vertices[vert_nf[c]]
                    
mesh.fill_holes()
mesh.process()
            
                
tic = time.time()
print("Total time:",round(tic-toc,3),"[s]")

Total time: 1.343 [s]


In [2]:
print(mesh)

<trimesh.Trimesh(vertices.shape=(68992, 3), faces.shape=(137960, 3))>


In [3]:
mesh_split = mesh.split()
print(mesh)
print(mesh_split)
print("len de mesh_split: ",len(mesh_split))

<trimesh.Trimesh(vertices.shape=(68992, 3), faces.shape=(137960, 3))>
[<trimesh.Trimesh(vertices.shape=(64818, 3), faces.shape=(129636, 3))>
 <trimesh.Trimesh(vertices.shape=(398, 3), faces.shape=(792, 3))>
 <trimesh.Trimesh(vertices.shape=(1552, 3), faces.shape=(3100, 3))>
 <trimesh.Trimesh(vertices.shape=(652, 3), faces.shape=(1300, 3))>
 <trimesh.Trimesh(vertices.shape=(352, 3), faces.shape=(700, 3))>
 <trimesh.Trimesh(vertices.shape=(200, 3), faces.shape=(396, 3))>
 <trimesh.Trimesh(vertices.shape=(1020, 3), faces.shape=(2036, 3))>]
len de mesh_split:  7


In [4]:
mesh_ses = mesh_split[0]
print(mesh_ses)

<trimesh.Trimesh(vertices.shape=(64818, 3), faces.shape=(129636, 3))>


In [5]:
mesh_ses.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [6]:
mesh_ses_faces = mesh_ses.faces + 1
mesh_ses_vert = mesh_ses.vertices
print(mesh_ses_faces, mesh_ses_vert)

[[ 1031  1020   805]
 [51062 55493 41147]
 [47362 55493 49088]
 ..., 
 [44020  2104 24282]
 [44020  5370  2104]
 [44019 36234 22343]] [[ 20.48   24.399  -4.827]
 [ 51.2    23.432   2.53 ]
 [ 51.2    36.861  14.336]
 ..., 
 [ 20.963  38.645  10.732]
 [ 20.963  33.427  16.364]
 [ 31.203  31.316 -16.19 ]]


In [7]:
np.savetxt("Meshing_1fni/1fni_split/1fni_d16_split.face",mesh_ses_faces,fmt="%i")
np.savetxt("Meshing_1fni/1fni_split/1fni_d16_split.vert",mesh_ses_vert)

In [8]:
#mesh: face - vert 1a0j
#d01: 7384 - 3690
#d02: 15684 - 7842
#d04: 32524 - 16260
#d08: 66580 - 33286
#d16: 135644 - 67816

#mesh: face - vert 1smd
#d01: 13896 - 6944
#d02: 31080 - 14650
#d04: 61240 - 30616
#d08: 125016 - 62500
#d16: 253844 - 135878

#mesh: face - vert 1fni
#d01: 7024 - 3514
#d02: 15052 - 7528
#d04: 30904 - 15454
#d08: 63756 - 31880
#d16: 129636 - 64818